# Synthetic Data Experiments

In [1]:
include("./src/NCP/NCP.jl");
include("./src/Misc.jl");

In [2]:
using .Misc
import .NCP

In [3]:
using Distributions, SpecialFunctions
using Combinatorics
import Dates: now

In [4]:
using Random
Random.seed!(1605); 

In [5]:
PATH = "./results/synt";
mkpath(PATH);

In [6]:
import Suppressor: @suppress_err

## On Hyperparameters of BAM

\begin{equation}
X^{(1)} = \left(\begin{array}{cccc} 
    2 & 1 & 1 & 0 \\
    0 & 0 & 1 & 2 \\
    0 & 0 & 1 & 1 
    \end{array} \right)
\end{equation}

In [7]:
R = 3
As = [10, 1e-1, 1e-10];

In [8]:
X = [2 1 1 0; 
     0 0 1 2;
     0 0 1 1];

In [9]:
@time log_PS_exact = [NCP.alloc_dist(X,R;a=a) for a ∈ As]
@time dₑₚ = NCP.EP_dist(X,R)

log_PX = map(logsumexp, log_PS_exact);

  5.717277 seconds (21.83 M allocations: 1.025 GiB, 5.57% gc time)
  0.384939 seconds (2.70 M allocations: 127.659 MiB, 8.95% gc time)


In [10]:
save_json("$PATH/effective_params.json"; X=X, R=R, a=As,
                        log_PS=log_PS_exact, EP=dₑₚ, time=now());

## Comparison of SMC, VB and exact enumeration on toy data

In [11]:
As = 10.0 .^ LinRange(-5,5,11)
Rs = 1:4;
EPOCHS, P, M = 100, 3000, 200;

\begin{equation}
X^{(1)} = \left(\begin{array}{cccc} 
    2 & 1 & 1 & 0 \\
    0 & 0 & 1 & 2 \\
    0 & 0 & 1 & 1 
    \end{array} \right)
\end{equation}

In [12]:
X = [2 1 1 0; 
     0 0 1 2;
     0 0 1 1];

In [13]:
@time log_PX_exact = [NCP.log_marginal(X,R; a=a) for R ∈ Rs, a ∈ As];
log_PR_exact = log_PX_exact .- logsumexp(log_PX_exact,dims=1);

 65.616628 seconds (571.00 M allocations: 25.857 GiB, 8.79% gc time)


In [14]:
@time log_PX_smc = logmeanexp([NCP.smc_weight(X,R,P; a=a)[1] for R ∈ Rs, a ∈ As, m ∈ 1:M],3);
log_PR_smc = log_PX_smc .- logsumexp(log_PX_smc,dims=1);

1117.911606 seconds (5.93 G allocations: 180.404 GiB, 2.44% gc time)


In [15]:
@time log_PX_vb = nanmax([NCP.standard_VB(X,R; a=a, EPOCHS=EPOCHS)[1][end] for R ∈ Rs, a ∈ As, m ∈ 1:M],3);
log_PR_vb = log_PX_vb .- logsumexp(log_PX_vb,dims=1);

 27.989278 seconds (236.66 M allocations: 14.647 GiB, 9.74% gc time)


In [16]:
save_json("$PATH/marginal_lkhd.json"; X=X, R=Rs, a=As, 
    log_PX=log_PX_exact, log_PX_smc=log_PX_smc, log_PX_vb=log_PX_vb, time=now());

\begin{equation}
X^{(2)} = \left(\begin{array}{ccc} 
    4 & 3 & 0 \\
    0 & 0 & 3 \\
    0 & 0 & 3 
    \end{array} \right)
\end{equation}

In [17]:
X = [4 3 0; 
     0 0 3;
     0 0 3];

In [18]:
@time log_PX_exact = [NCP.log_marginal(X,R; a=a) for R ∈ Rs, a ∈ As];
log_PR_exact = log_PX_exact .- logsumexp(log_PX_exact,dims=1);

167.375861 seconds (1.34 G allocations: 59.756 GiB, 8.16% gc time)


In [19]:
@time log_PX_smc = logmeanexp([NCP.smc_weight(X,R,P; a=a)[1] for R ∈ Rs, a ∈ As, m ∈ 1:M],3);
log_PR_smc = log_PX_smc .- logsumexp(log_PX_smc,dims=1);

1588.249755 seconds (8.42 G allocations: 251.492 GiB, 2.10% gc time)


In [20]:
@time log_PX_vb = nanmax([NCP.standard_VB(X,R; a=a, EPOCHS=EPOCHS)[1][end] for R ∈ Rs, a ∈ As, m ∈ 1:M],3);
log_PR_vb = log_PX_vb .- logsumexp(log_PX_vb,dims=1);

 23.242315 seconds (204.52 M allocations: 12.184 GiB, 7.23% gc time)


In [21]:
save_json("$PATH/marginal_lkhd2.json"; X=X, R=Rs, a=As, 
    log_PX=log_PX_exact, log_PX_smc=log_PX_smc, log_PX_vb=log_PX_vb, time=now());

## Posterior Distribution of the number of tokens $S_+$

\begin{equation}
X^{(3)} = \left(\begin{array}{cc} 
    3 & ? \\
    3 & 3 \\ 
    \end{array} \right)
\end{equation}

In [22]:
X_miss = [3 NaN; 
          3 3];

In [23]:
Rs = 1:4
a, b = 4.0, 1e-2

Ts = nansum(X_miss):15;

In [24]:
@time log_PT_exact = [NCP.log_posterior_T(X_miss, T, R; a=a, b=b) for R ∈ Rs, T ∈ Ts];
@time log_PT_smc = [NCP.log_posterior_T(X_miss, T, R; a=a, b=b, smc=true) for R ∈ Rs, T ∈ Ts];
@time log_PT_vb = [NCP.log_posterior_T(X_miss, T, R; a=a, b=b, elbo=true) for R ∈ Rs, T ∈ Ts];

126.143116 seconds (731.21 M allocations: 27.432 GiB, 3.74% gc time)
  2.366245 seconds (9.66 M allocations: 318.291 MiB, 2.40% gc time)
  7.770400 seconds (52.98 M allocations: 2.782 GiB, 6.30% gc time)


In [25]:
save_json("$PATH/Sp_dist.json"; X=X_miss, R=Rs, a=a, b=b, T=Ts,
    log_PT=log_PT_exact, log_PT_smc=log_PT_smc, log_PT_vb=log_PT_vb, time=now());

\begin{equation}
X^{(4)} = \left(\begin{array}{ccc} 
    4 & ? \\
    4 & 1
    \end{array} \right)
\end{equation}

In [26]:
X_miss = [4 NaN; 
          4 1];

In [27]:
Rs = 1:4
a, b = 4.0, 1e-2

Ts = nansum(X_miss):15;

In [28]:
@time log_PT_exact = [NCP.log_posterior_T(X_miss, T, R; a=a, b=b) for R ∈ Rs, T ∈ Ts];
@time log_PT_smc = [NCP.log_posterior_T(X_miss, T, R; a=a, b=b, smc=true) for R ∈ Rs, T ∈ Ts];
@time log_PT_vb = [NCP.log_posterior_T(X_miss, T, R; a=a, b=b, elbo=true) for R ∈ Rs, T ∈ Ts];

 76.222895 seconds (447.05 M allocations: 16.736 GiB, 3.93% gc time)
  1.746150 seconds (8.54 M allocations: 265.086 MiB, 2.60% gc time)
  6.762978 seconds (51.14 M allocations: 2.699 GiB, 7.00% gc time)


In [29]:
save_json("$PATH/Sp_dist2.json"; X=X_miss, R=Rs, a=a, b=b, T=Ts,
    log_PT=log_PT_exact, log_PT_smc=log_PT_smc, log_PT_vb=log_PT_vb, time=now());

## Optimized Implementations for CP/PARAFAC

### Variational Bayes

In [30]:
function vb_opt(X::Array{ℜ,3}, R::Ƶ; a::ℜ=1.0, b::ℜ=a/nansum(X), EPOCHS::Ƶ=1, ϵ::ℜ=1e-16) where {ℜ<:Real, Ƶ<:Int}
    I::Ƶ, J::Ƶ, K::Ƶ = size(X)
    α_R, α_IR, α_JR, α_KR = fill(a/R,R), fill(a/(I*R),I,R), fill(a/(J*R),J,R), fill(a/(K*R),K,R)
    
    log_λ = log(rand(Gamma(a,1.0/b)))
    log_θ_R = rand(Dirichlet(α_R .+ 1.0/R))
    log_θ_IR = reshape([log(θ_ir) for r=1:R for θ_ir=rand(Dirichlet(α_IR[:,r] .+ 1.0/I))],I,R)
    log_θ_JR = reshape([log(θ_jr) for r=1:R for θ_jr=rand(Dirichlet(α_JR[:,r] .+ 1.0/J))],J,R)
    log_θ_KR = reshape([log(θ_kr) for r=1:R for θ_kr=rand(Dirichlet(α_KR[:,r] .+ 1.0/K))],K,R)
    
    s, S_R, S_IR, S_JR, S_KR, S₊ = zeros(R), zeros(R), zeros(I,R), zeros(J,R), zeros(K,R), sum(X)
    log_p = zeros(R)

    ELBO = a*log(b) - (a + S₊)*log(b + 1) + lgamma(a + S₊) - lgamma(a) - sum(lgamma, X .+ 1)                  

    for eph=1:EPOCHS
        S_R .= 0.0
        S_IR .= 0.0
        S_JR .= 0.0
        S_KR .= 0.0
                                        
        for k=1:K, j=1:J, i=1:I #order of traversal is important
            log_p .= log_θ_R .+ log_θ_IR[i,:] .+ log_θ_JR[j,:] .+ log_θ_KR[k,:]
            log_p .-= logsumexp(log_p)
            
            s .= X[i,j,k] .* exp.(log_p)
            
            S_R .+= s
            S_IR[i,:] .+= s
            S_JR[j,:] .+= s
            S_KR[k,:] .+= s
                                                            
            if eph == EPOCHS
                ELBO -= sum(s .* log_p)
            end
        end
                                                    
        log_λ = digamma(S₊+a) - log(b+1.0)
        log_θ_R .= digamma.(S_R.+α_R) .- digamma(S₊+a)
        log_θ_IR .= digamma.(S_IR.+α_IR) .- digamma.(S_R.+α_R)'
        log_θ_JR .= digamma.(S_JR.+α_JR) .- digamma.(S_R.+α_R)'
        log_θ_KR .= digamma.(S_KR.+α_KR) .- digamma.(S_R.+α_R)'
        
        if eph == EPOCHS                                         
            ELBO += sum(lbeta(α_IR .+ S_IR;dims=1)) - sum(lbeta(α_IR;dims=1))
            ELBO += sum(lbeta(α_JR .+ S_JR;dims=1)) - sum(lbeta(α_JR;dims=1))
            ELBO += sum(lbeta(α_KR .+ S_KR;dims=1)) - sum(lbeta(α_KR;dims=1))
            ELBO += lbeta(α_R .+ S_R) - lbeta(α_R)
        end
    end
    return ELBO
end

vb_opt (generic function with 1 method)

### Sequential Monte Carlo

In [31]:
import DataStructures: PriorityQueue, peek, enqueue!, dequeue!
import Base: iterate, length, sum, eltype

In [32]:
mutable struct EventQueue{ℜ<:Real} 
    T::Int
    X::Array{ℜ}
    pq::PriorityQueue{Tuple{Int,Int,Int},ℜ,Base.Order.ForwardOrdering}
    function EventQueue(X::Array{ℜ}) where {ℜ<:Real}
        I::Int, J::Int, K::Int = size(X)
        T::Int = Int(round(sum(X)))
        pq = PriorityQueue([(i,j,k)=>rand(Dirichlet([1.0,X[i,j,k]]))[1] for k=1:K, j=1:J, i=1:I if X[i,j,k]>0])
        return new{ℜ}(T,copy(X),pq)
    end
end
                    
Base.sum(L::EventQueue) = sum(L.X)
Base.length(L::EventQueue) = L.T
Base.eltype(L::EventQueue) = Tuple{Int,Int,Int}

function Base.iterate(L::EventQueue{ℜ}, state::Ƶ=1) where {Ƶ<:Int,ℜ<:Real}
    i::Ƶ, j::Ƶ, k::Ƶ = 0, 0, 0
    t::ℜ, t_next::ℜ = 0.0, 0.0
    if L.T < state
        return nothing
    end
    t = peek(L.pq)[2]
    i, j, k = dequeue!(L.pq)
    L.X[i, j, k] -= 1.0
    if L.X[i, j, k] > 0.0
        t_next = t + (1.0 - t)*rand(Dirichlet([1.0, L.X[i,j, k]]))[1]
        L.pq[(i, j, k)] = t_next
    end
    return (i, j, k), state+1
end

In [33]:
mutable struct Particle{ℜ <: Real}
    S_R::Array{ℜ,1}
    S_IR::Array{ℜ,2}
    S_JR::Array{ℜ,2}
    S_KR::Array{ℜ,2}
    function Particle(I::Int,J::Int, K::Int, R::Int; γ::ℜ=0.1) where {ℜ<:Real}
        a::ℜ = I*J*K*γ
        return new{ℜ}(fill(a/R,R), fill(a/(R*I),I,R), fill(a/(R*J),J,R), fill(a/(R*K),K,R))
    end
    function Particle(S_R::Array{ℜ,1}, S_IR::Array{ℜ,2}, S_JR::Array{ℜ,2}, S_KR::Array{ℜ,2}) where {ℜ<:Real}
        return new{ℜ}(S_R, S_IR, S_JR, S_KR)
    end
end

In [34]:
function resample(W::Array,Π::Array{Particle{ℜ}},u=rand()) where {ℜ <: Real} # systematic resampling
    P = length(W)
    j = 0
    cum_Wⱼ = cum_Wᵢ = -u
    for i ∈ 1:P
        rᵢ = ceil(cum_Wᵢ + P*W[i]) - ceil(cum_Wᵢ) # number of replicas for ith particle
        for _ ∈ 2.0:rᵢ
            j+=1
            while ceil(cum_Wⱼ+ P*W[j]) - ceil(cum_Wⱼ) > 0 # find next j to be replaced
                cum_Wⱼ += P*W[j]
                j+=1
            end
            cum_Wⱼ += P*W[j]
            
            # replace j by i
            Π[j].S_R .= Π[i].S_R
            Π[j].S_IR .= Π[i].S_IR
            Π[j].S_JR .= Π[i].S_JR
            Π[j].S_KR .= Π[i].S_KR          
        end
        cum_Wᵢ += P*W[i]
    end
end

function smc_opt(X::Array{ℜ,3}, R::Ƶ, N::Ƶ=1; a::ℜ=1.0, b::ℜ=a/nansum(X)) where {ℜ<:Real, Ƶ<:Int}
    I::Ƶ, J::Ƶ, K::Ƶ = size(X)
    T::Ƶ = Ƶ(sum(X))

    log_Z::ℜ = a*log(b) - (a+T)*log(b + 1.0) - sum(lgamma,X .+ 1.0) 
    P = [Particle(I,J,K,R; γ=a/(I*J*K)) for n=1:N]
    
    log_w::Array{ℜ}, W::Array{ℜ}, cum_W::Array{ℜ} = fill(log_Z,N), fill(1.0/N,N), zeros(ℜ,N)
    log_ν::ℜ, log_q::Array{ℜ}, q::Array{ℜ}= 0.0, zeros(ℜ,R), zeros(ℜ,R)

    for (i, j, k) ∈ EventQueue(X)
        for (n,p) ∈ enumerate(P)   
            log_q .= log.(p.S_IR[i,:]) .+ log.(p.S_JR[j,:]) .+ log.(p.S_KR[k,:]) .- 2.0.*log.(p.S_R)
            log_ν = logsumexp(log_q)
            log_q .-= log_ν
            
            q .= exp.(log_q)
            r = rand(Categorical(q))
            
            p.S_R[r] += 1.0 
            p.S_IR[i,r] += 1.0 
            p.S_JR[j,r] += 1.0 
            p.S_KR[k,r] += 1.0 

            log_w[n] += log_ν
        end
        
        log_Z = logmeanexp(log_w)
        W .= exp.(log_w .- logsumexp(log_w))
        
        if 2/N < sum(W .* W)
            resample(W,P)
            log_w .= log_Z
        end
    end
    return log_Z
end

smc_opt (generic function with 2 methods)

## Model Order Selection for CP/PARAFAC

### Confusion Matrix

In [35]:
import Random: shuffle
import Dates: now

In [36]:
I, J, K = 20, 25, 30
T = 1000
a = 10.0

10.0

In [37]:
N, M = 1000, 20
EPOCHS = 50
Rs = 1:8

1:8

In [38]:
R_true = shuffle(vcat([Rs for i ∈ 1:15]...))
R_smc, R_vb = zeros(Int,length(R_true)), zeros(Int,length(R_true));

In [39]:
for (t,Rₜ) ∈ enumerate(R_true)
    X = Float64.(NCP.generate(T,Rₜ,I,J,K; a=a)[1]);
    _, R_smc[t] = findmax(logmeanexp([smc_opt(X, R, N; a=a) for m ∈ 1:M, R ∈ Rs],1))
    _, R_vb[t] = findmax(nanmax([vb_opt(X, R; a=a, EPOCHS=EPOCHS) for m ∈ 1:M, R ∈ Rs],1))
end

In [40]:
save_json("$PATH/parafac_confusion.json"; R = Rs, a=As, I=I, J=J, K=K, T=T,
    R_true=R_true, R_smc=R_smc, R_vb=R_vb, time=now());

### An Example

In [41]:
I, J, K, R_true = 20, 25, 30, 5
a, T = 10.0, 1000

(10.0, 1000)

In [42]:
X = Float64.(NCP.generate(T,R_true,I,J,K;a=a)[1]);

In [43]:
N, M = 1500, 40
EPOCHS = 50
Rs = 1:10;

In [44]:
@time smc_results = logmeanexp([smc_opt(X, R, N; a=a) for m ∈ 1:M, R ∈ Rs],1)
@time vb_results = nanmax([vb_opt(X, R; a=a, EPOCHS=EPOCHS) for m ∈ 1:M, R ∈ Rs],1);

513.203926 seconds (3.04 G allocations: 320.986 GiB, 7.04% gc time)
213.838334 seconds (2.15 G allocations: 252.273 GiB, 17.65% gc time)


In [45]:
save_json("$PATH/parafac_model_selection.json"; R = Rs, a=a, I=I, J=J, K=K, R_true=R_true, T=T,
    log_PX_smc=smc_results, log_PX_vb=vb_results,  time=now());

### Runtimes

In [46]:
IJK = [(Iₙ,Iₙ,Iₙ) for Iₙ ∈ 4:4:64]
a, T, R = 50.0, 1000, 10
EPOCHS, P, M = 50, 500, 10

(50, 500, 10)

In [47]:
smc_times = zeros(Real,length(IJK),M)
vb_times = zeros(Real,length(IJK),M)
for m ∈ 1:M, (i,(I,J,K)) in enumerate(IJK)
    X = Float64.(NCP.generate(T,R,I,J,K; a=a)[1])
    
    res = @timed smc_opt(X, R, P; a=a);
    smc_times[i,m] = res[2]
    res = @timed vb_opt(X, R; a=a, EPOCHS=EPOCHS);
    vb_times[i,m] = res[2]
end

In [48]:
save_json("$PATH/parafac_runtime.json"; R = R, a=a, T=T, IJK=IJK, EPOCHS=EPOCHS, P=P, M=M,
    smc=smc_times, vb=vb_times,  time=now());